In [229]:
import numpy as np
import pandas as pd

In [230]:
df = pd.read_csv('enron_spam_data.csv')

In [231]:
df.sample(3)

,Message ID,Subject,Message,Spam/Ham,Date
9363,9363,alp presentation,"christie ,\nshirley reserved room 49 cl for monday 4 : 00 p . m . presentation .\ncan you issue the formal invitation to our guests with the game / dinner\ndetails ? i don ' t have all the details regarding the enron field\nbox and time . i am out most of the day on wednesday but we can\ndiscuss the details on thursday .\nhope to see you on saturday at the concert .\nvince",ham,2001-05-01
1005,1005,devon transition,"effective july 1 avails , beverly beaty will be enron ' s lead operations\ncontact for devon production . please assemble all information and contact\nall personnel necessary to make the transition as smooth as possible .\nto eliminate unnecesary e - mails , please designate one main contact from each\nof your desks and a backup ( central desk , west desk and texas desk ) who can\nreceive and distribute all nom changes and operational information to your\ndesk ' s proper schedulers . if you have any questions , please let me know .\nthanks !",ham,2000-06-19
7301,7301,re :,"life . . . : )\nmaybe next time .\nroman\n- - - - - original message - - - - -\nfrom : vince . j . kaminski @ enron . com\nto : rkosecki @ mieco . com\nsent : 10 / 18 / 00 1 : 14 pm\nsubject : re :\nroman ,\nsorry . i am leaving for philadelphia this evening . leaving office around\n5\np . m . let ' s get together on another occasion .\nvince\nroman kosecki on 10 / 18 / 2000 02 : 09 : 28 pm\nto : "" ' vince . j . kaminski @ enron . com ' ""\ncc :\nsubject : re :\nhi vince ,\ni am in houston now . will have to work late . but how about\ndrinks / food\nsome time around 7 - 8 or later ? i am staying at wyndham greenplace ( ? )\nand\nwill use taxi to get around .\nthanks\nroman\n- - - - - original message - - - - -\nfrom : vince . j . kaminski @ enron . com\nto : rkosecki @ mieco . com\nsent : 10 / 13 / 00 7 : 55 am\nsubject : re :\nroman\nthanks .\nmy home number is 281 367 5377\nvince\nroman kosecki on 10 / 13 / 2000 09 : 38 : 39 am\nto : "" ' vince . j . kaminski @ enron . com ' ""\ncc :\nsubject : re :\ni will be in houston on wed . willl give you a call\nroman\n- - - - - original message - - - - -\nfrom : vince . j . kaminski @ enron . com [ mailto : vince . j . kaminski @ enron . com ]\nsent : thursday , october 12 , 2000 2 : 43 pm\nto : rkosecki @ mieco . com\nsubject : re :\nroman ,\ndrinks after work would be better . i am flying back from ca on tue\nmorning .\nplease , call me at 713 853 3848 or 713 410 5396 ( cell )\nin the afternoon .\nvince\nroman kosecki on 10 / 12 / 2000 12 : 21 : 50 pm\nto : "" ' vince . j . kaminski @ enron . com ' ""\ncc :\nsubject :\nhi vince\ni will be in houston for a few days next week ( like mon - wed ) . if you\nare\nback from your european trips maybe we can "" do "" lunch ?\nroman",ham,2000-10-18


In [232]:
df.shape

(33716, 5)

In [233]:
!pip install nltk


[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [234]:
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\suraj\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\suraj\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [235]:
#1 Text Cleaning Methods 

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def clean_email(text):
    
    #1 Remove email headers (Message-ID, Date, etc.)
    text = re.sub(r'^(Message-ID|Date|From|To|Subject):.*\n?', '', text, flags=re.MULTILINE)
    
    #2 Remove MIME boundaries
    text = re.sub(r'--\S+', '', text)
    
    #3 Remove HTML tags
    text = re.sub(r'<.*?>', ' ', text)
    
    #4 Replace URLs with token
    text = re.sub(r'http\S+|www\S+', ' <URL> ', text)
    
    #5 Replace email addresses
    text = re.sub(r'\S+@\S+', ' <EMAIL> ', text)
    
    #6️ Convert to lowercase
    text = text.lower()
    
    #7️ Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    #8️ Remove digits (optional)
    text = re.sub(r'\d+', '', text)
    
    #9️ Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    
    #10 Tokenization
    tokens = text.split()
    
    #11 Remove stopwords + short words + lemmatize
    cleaned_tokens = []
    for word in tokens:
        if word not in stop_words and len(word) > 2:
            word = lemmatizer.lemmatize(word)
            cleaned_tokens.append(word)
    
    return " ".join(cleaned_tokens)

In [236]:
df['combined_text'] = df['Subject'].fillna('') + " " + df['Message'].fillna('')
df['cleaned_text'] = df['combined_text'].apply(clean_email)

In [237]:
pd.set_option('display.max_colwidth', None)
df[['combined_text', 'cleaned_text']].head(2)

combined_text  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              

In [238]:
#Train-Test Split (80-20 ratio)

from sklearn.model_selection import train_test_split

X = df['cleaned_text']
y = df['Spam/Ham']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [250]:
#2 TF-IDF Vectorization (Tuned with N-Grams features)

from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(
    max_features=10000,
    ngram_range=(1, 2),      # Unigrams + Bigrams
    min_df=5,                # Ignore rare noise words
    max_df=0.85,              # Remove overly common words
    sublinear_tf=True,       # Better scaling
)

X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

In [251]:
vectorizer.get_feature_names_out()[:10]

array(['abacha', 'abandoned', 'abb', 'abc', 'abel', 'abidjan', 'abiiity',
       'ability', 'able', 'able get'], dtype=object)

In [252]:
# Model 1 - Naive Bayes
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Initialize model
nb_model = MultinomialNB()

# Train
nb_model.fit(X_train_tfidf, y_train)

# Predict
nb_pred = nb_model.predict(X_test_tfidf)

# Accuracy
nb_accuracy = accuracy_score(y_test, nb_pred)
print("Naive Bayes Accuracy:", nb_accuracy)

# Classification Report
print("\nNaive Bayes Classification Report:\n")
print(classification_report(y_test, nb_pred))

# Confusion Matrix
print("Naive Bayes Confusion Matrix:\n")
print(confusion_matrix(y_test, nb_pred))

Naive Bayes Accuracy: 0.9859134045077106

Naive Bayes Classification Report:

              precision    recall  f1-score   support

         ham       0.99      0.98      0.99      3309
        spam       0.98      0.99      0.99      3435

    accuracy                           0.99      6744
   macro avg       0.99      0.99      0.99      6744
weighted avg       0.99      0.99      0.99      6744

Naive Bayes Confusion Matrix:

[[3255   54]
 [  41 3394]]


In [253]:
# Model 2 - Logistic Regression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Initialize model
lr_model = LogisticRegression(max_iter=1000)

# Train
lr_model.fit(X_train_tfidf, y_train)

# Predict
lr_pred = lr_model.predict(X_test_tfidf)

# Accuracy
lr_accuracy = accuracy_score(y_test, lr_pred)
print("Logistic Regression Accuracy:", lr_accuracy)

# Classification Report
print("\nLogistic Regression Classification Report:\n")
print(classification_report(y_test, lr_pred))

# Confusion Matrix
print("Logistic Regression Confusion Matrix:\n")
print(confusion_matrix(y_test, lr_pred))

Logistic Regression Accuracy: 0.9905100830367735

Logistic Regression Classification Report:

              precision    recall  f1-score   support

         ham       1.00      0.98      0.99      3309
        spam       0.98      1.00      0.99      3435

    accuracy                           0.99      6744
   macro avg       0.99      0.99      0.99      6744
weighted avg       0.99      0.99      0.99      6744

Logistic Regression Confusion Matrix:

[[3256   53]
 [  11 3424]]


In [254]:
# Model 3 - Linear Support Vector Machine (Linear SVM)
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV

# Base linear SVM
base_svm = LinearSVC(C=0.3, class_weight={'ham': 2, 'spam': 1}, max_iter=10000)

# Calibrated version (adds predict_proba)
svm_model = CalibratedClassifierCV(
    base_svm,
    method='isotonic',
    cv=5
)

# Train
svm_model.fit(X_train_tfidf, y_train)

# Predict
svm_pred = svm_model.predict(X_test_tfidf)

# Accuracy
svm_accuracy = accuracy_score(y_test, svm_pred)
print("Linear SVM Accuracy:", svm_accuracy)

# Classification Report
print("\nSVM Classification Report:\n")
print(classification_report(y_test, svm_pred))

# Confusion Matrix
print("SVM Confusion Matrix:\n")
print(confusion_matrix(y_test, svm_pred))

Linear SVM Accuracy: 0.9934756820877817

SVM Classification Report:

              precision    recall  f1-score   support

         ham       0.99      0.99      0.99      3309
        spam       0.99      0.99      0.99      3435

    accuracy                           0.99      6744
   macro avg       0.99      0.99      0.99      6744
weighted avg       0.99      0.99      0.99      6744

SVM Confusion Matrix:

[[3288   21]
 [  23 3412]]


In [255]:
def predict_email_ensemble(text):
    
    # ---------- Preprocess ----------
    cleaned = clean_email(text)
    vectorized = vectorizer.transform([cleaned])
    
    # ---------- Model Predictions ----------
    nb_pred = str(nb_model.predict(vectorized)[0])
    lr_pred = str(lr_model.predict(vectorized)[0])
    svm_pred = str(svm_model.predict(vectorized)[0])
    
    # ---------- Probabilities ----------
    
    # Naive Bayes
    nb_prob = nb_model.predict_proba(vectorized)[0]
    nb_spam_index = list(nb_model.classes_).index('spam')
    nb_spam_conf = nb_prob[nb_spam_index]
    
    # Logistic Regression
    lr_prob = lr_model.predict_proba(vectorized)[0]
    lr_spam_index = list(lr_model.classes_).index('spam')
    lr_spam_conf = lr_prob[lr_spam_index]
    
    # Calibrated Linear SVM
    svm_prob = svm_model.predict_proba(vectorized)[0]
    svm_spam_index = list(svm_model.classes_).index('spam')
    svm_spam_conf = svm_prob[svm_spam_index]
    
    # ---------- Weighted Soft Voting ----------
    final_spam_score = (
        0.4 * nb_spam_conf +
        0.4 * lr_spam_conf +
        0.2 * svm_spam_conf
    )
    
    final_pred = 'spam' if final_spam_score >= 0.7 else 'ham'
    
    return {
        "Naive Bayes": nb_pred,
        "NB Spam Confidence": round(nb_spam_conf * 100),
        "Logistic Regression": lr_pred,
        "LR Spam Confidence": round(lr_spam_conf * 100),
        "Linear SVM": svm_pred,
        "SVM Spam Confidence": round(svm_spam_conf * 100),
        "Final Spam Score": round(final_spam_score * 100),
        "Final Decision": final_pred.upper()
    }

In [246]:
# Random Tests :-

email_text = """
URGENT!!! You have won $10,000.
Click the link below to claim now.
"""

result = predict_email_ensemble(email_text)
print(result)

{'Naive Bayes': 'spam', 'NB Spam Confidence': 96, 'Logistic Regression': 'spam', 'LR Spam Confidence': 96, 'Linear SVM': 'spam', 'SVM Spam Confidence': 100, 'Final Spam Score': 97, 'Final Decision': 'SPAM'}


In [256]:
email_text = """
Congratulations!!! You have won $5000.
Click here to claim your prize now.
"""

result = predict_email_ensemble(email_text)
print(result)

{'Naive Bayes': 'spam', 'NB Spam Confidence': 92, 'Logistic Regression': 'spam', 'LR Spam Confidence': 84, 'Linear SVM': 'spam', 'SVM Spam Confidence': 94, 'Final Spam Score': 89, 'Final Decision': 'SPAM'}


In [257]:
email_text = """
The Team is impressed by your marketing strategy. Let's catch up sometime for disussing a marketing campaign !
- Marketing Head
"""

result = predict_email_ensemble(email_text)
print(result)

{'Naive Bayes': 'ham', 'NB Spam Confidence': 22, 'Logistic Regression': 'spam', 'LR Spam Confidence': 69, 'Linear SVM': 'spam', 'SVM Spam Confidence': 94, 'Final Spam Score': 55, 'Final Decision': 'HAM'}


In [258]:
email_text = """
It's nice connecting with you. Your refferal really helped a lot. Thanks for your support. 
"""

result = predict_email_ensemble(email_text)
print(result)

{'Naive Bayes': 'spam', 'NB Spam Confidence': 63, 'Logistic Regression': 'ham', 'LR Spam Confidence': 39, 'Linear SVM': 'ham', 'SVM Spam Confidence': 12, 'Final Spam Score': 43, 'Final Decision': 'HAM'}
